In [4]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
import dask_cudf as pd
import cudf
cluster = LocalCUDACluster(CUDA_VISIBLE_DEVICES="0")
client = Client(cluster)
client.run(cudf.set_allocator, "managed")
client


#lol numpy


/home/karl/anaconda3/envs/rapids-21.08/lib/python3.7/site-packages/distributed/node.py:161: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36073 instead
  f"Port {expected} is already in use.\n"
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/home/karl/anaconda3/envs/rapids-21.08/lib/python3.7/multiprocessing/spawn.py", line 105, in spawn_main
    exitcode = _main(fd)
  File "/home/karl/anaconda3/envs/rapids-21.08/lib/python3.7/multiprocessing/spawn.py", line 115, in _main
    self = reduction.pickle.load(from_parent)
ModuleNotFoundError: No module named 'dask_cuda'
distributed.nanny - WARNING - Restarting worker
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/home/karl/anaconda3/envs/rapids-21.08/lib/python3.7/multiprocessing/spawn.py", line 105, in spawn_main
    exitcode = _main(fd)
  File "/home/karl/anaconda3/envs/rapids-21.08/lib/python3.7/m

Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:36073/status,
Dashboard: http://127.0.0.1:36073/status,Workers: 0
Total threads: 0,Total memory: 0 B
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33095,Workers: 0
Dashboard: http://127.0.0.1:36073/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [17]:
#columns from MRCONSO file in UMLS
#note for this file I have to rename the AUI column to AUI2 - this is because it is necessary for final merge
mrconso_columns = [
    "CUI",
    "LAT",
    "TS",
    "LUI",
    "STT",
    "SUI",
    "ISPREF",
    "AUI2",
    "SAUI",
    "SCUI",
    "SDUI",
    "SAB",
    "TTY",
    "CODE",
    "STR",
    "SRL",
    "SUPRESS",
    "CVF",
]

#dask_cudf can read in any tab delimited file with a similar command as pandas
mrconso_iter = pd.read_csv(
    "/home/karl/PycharmProjects/doctorlingo/test_scripts/cwi-master/CWI_Sequence_Labeller/datasets/UMLS/MRCONSO.RRF",
    sep="|",
    names=mrconso_columns,
    #chunksize="1 GB"
)
mrconso_iter
mrconso = mrconso_iter[mrconso_iter["LAT"] == "ENG"]
# mrconso = mrconso.map_partitions(lambda df: df.to_pandas())
mrconso

,CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI2,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPRESS,CVF
npartitions=8,,,,,,,,,,,,,,,,,,
,object,object,object,object,object,object,object,object,float64,object,object,object,object,object,object,int64,object,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [10]:
# below dataframe for short is mmmhdf
merged_mrconso_mrrel_hypernymy_columns = [
    "CUI",
    "STR",
    "AUI",
    "AUI2"

]


mmmhdf = pd.read_csv(
    "mrconso_mrrel_hypernym_first_merge.csv",
    names=merged_mrconso_mrrel_hypernymy_columns,
    chunksize="1 GB"
)


# mrrel = mrrel.map_partitions(lambda df: df.to_pandas())



In [11]:
mrconso = mrconso[["AUI2", "STR2"]]
#mrconso.compute()
#mrconso = mrconso.drop_duplicates()
#mrconso.compute()

In [12]:
#o_partitions = max(mmmhdf.npartitions, mrconso.npartitions)  * 2
merged_df = mmmhdf.merge(mrconso, how="left", on="AUI2")


In [15]:
#merged_df.compute()#.to_pandas()

merged_df.to_csv('mrconso_mrrel_hypernym_final_merge.csv', sep='|', single_file=True)

['/home/karl/PycharmProjects/pythonProject/buildNERhypernymy/mrconso_mrrel_hypernym_final_merge.csv']

In [ ]:

"""

Concatenated rows exceeds size_type range after merge operation[BUG]

Solution:

You are running into this issue because the output merge of small chunks may be too big (> 2,147,483,648) .

This can happen if you have a lot of repeated keys as we repeat each row in the right table that matches the left table.
See the below example for an explanation.

Now mitigating this can be done in multiple ways :

1. If one of the frames is a single partitions (For broadcast joints) :
Repartition the multi partitioned table upfront into more partitions:

## the right frame is a single partition
assert ddf_2.npartitions == 1

merged_df = ddf_1 = ddf_1.repartition(npartitions=ddf_1.npartitions*2)
ddf_1.merge(ddf_2,how=inner)

2. If both of them have multiple partitions (Hash-Join)
Using npartitions: This argument controls the number of output partitions.

o_partitions = max(ddf_1.npartitions, ddf_2.npartitions )*2
merged_df = ddf_1.merge(ddf_2,npartitions= o_partitions,  how=inner)

It's tough to know how to mitigate this error without more information
like the number of left and right partitions but I hope that this still helps.

https://www.gitmemory.com/issue/rapidsai/cudf/8182/834834910

cudf memory error solutions:

First, if output can be saved to a csv or parquet file as a dataframe
Don't compute or to_pandas - just have final computations with to_csv
This will allow it to pipe straight to storage instead of using all mem

Try using RMM managed memory. 
You can enable that by setting the RMM allocator through a Dask client.run call:
i.e: the preamble to this script

client = Client(cluster)  # Sample client connecting to `cluster` object
client.run(cudf.set_allocator, "managed")  # Uses managed memory instead of "default"
By doing this, you'll be using CUDA unified memory, which will take care of moving data GPU<->CPU automatically when the GPU is running OOM.

EDIT: If you're not using Dask, you can simply use cudf.set_allocator("managed").

https://docs.rapids.ai/api/dask-cuda/nightly/index.html
https://dask-cuda.readthedocs.io/en/latest/specializations.html#spilling-from-device
"""